In [1]:
#import oboe modules
import sys
import pandas as pd
import os
automl_path = '../automl/'
sys.path.append(automl_path)
from auto_learner import AutoLearner
import util
import time
#import libraries
import numpy as np
import multiprocessing

#import scikit-learn modules
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
#experimental settings
VERBOSE = False #whether to print out information indicating current fitting progress
N_CORES = 1 #number of cores
RUNTIME_BUDGET = 15 #the actual runtime would be closest to budget if the budget is close to powers of 2

In [3]:
#optional: limit the types of algorithms
s = ['AB', 'ExtraTrees', 'GNB', 'KNN', 'RF', 'DT']

In [4]:
#load dataset for meta-training and meta-testing
data = load_iris()
x = np.array(data['data'])
y = np.array(data['target'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [5]:
#autolearner arguments
autolearner_kwargs = {
    'runtime_limit': RUNTIME_BUDGET,
    'verbose': VERBOSE,
    'selection_method': 'min_variance',
    'algorithms': s,
    'stacking_alg': 'greedy',
    'n_cores': N_CORES,
}

In [6]:
m = AutoLearner('classification', **autolearner_kwargs)

In [7]:
start = time.time()
m.fit_doubling_time_constrained(x_train, y_train)
elapsed_time = time.time() - start

../automl/convex_opt.py:50: RuntimeWarning: invalid value encountered in double_scalars
  return -1 * sign * log_det
/home/cy438/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1817: RuntimeWarning: invalid value encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)
../automl/auto_learner.py:129: RuntimeWarning: invalid value encountered in greater
  to_sample = valid[np.where(v_opt > 0.9)[0]]
/home/cy438/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [8]:
y_predicted = m.predict(x_test)
print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))    
print("elapsed time: {}".format(elapsed_time))

prediction error: 0.02876984126984128
elapsed time: 14.528945207595825
